<h1 align="center">Laboratorio: <span style="font-weight:normal">Uso reales de filtros espaciales y morfológicos</span></h1>

In [7]:
import numpy as np
import scipy.ndimage as ndi
import cv2
from datetime import datetime
from PIL import Image, ImageOps

In [8]:
# Implementación desarrollada por mi parte para poder distinguir la imagen con su implementación
def guardar_mostrar_imagen(imagen_procesada, titulo_imagen: str = "", nombre_imagen: str = ""):
    time = datetime.now()
    ruta_imagen_procesada = "img-fil-laplaciano/procesadas/" + nombre_imagen + "_" + time.strftime("%f") + ".jpg"

    cv2.putText(imagen_procesada, titulo_imagen, (3, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imwrite(ruta_imagen_procesada, imagen_procesada)
    img_2 = Image.open(ruta_imagen_procesada)
    img_2.show()

In [9]:
def high_pass_filter(img_in):
    # Write high pass filter here
    dft = cv2.dft(np.float32(img_in), flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    magnitude_spectrum = 20 * np.log(cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]))
    rows, cols = img_in.shape
    crow, ccol = rows / 2, cols / 2
    mask = np.ones((rows, cols, 2), np.uint8)
    mask[int(crow - 10):int(crow + 10), int(ccol - 10):int(ccol + 10)] = 0
    fshift = dft_shift * mask
    f_ishift = np.fft.ifftshift(fshift)
    img_back = cv2.idft(f_ishift, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT)
    img_out = img_back  # High pass filter result

    return img_out

In [10]:
def procesar_imagen_con_opencv(path_image: str):
    ddepth = cv2.CV_16S
    kernel_size = 3
    source = cv2.imread(path_image, cv2.IMREAD_COLOR)

    # Eliminación del ruido difuminado con el filtro gaussiano
    imagen_filtro_guassiano = cv2.GaussianBlur(source, (3, 3), 0)

    # Convertir la imagen a escala de grises
    imagen_escala_grises = cv2.cvtColor(imagen_filtro_guassiano, cv2.COLOR_BGR2GRAY)

    # Aplicación de la función Laplace para el realce de bordes
    imagen_procesada = cv2.Laplacian(imagen_escala_grises, ddepth, ksize=kernel_size)
    imagen_procesada = cv2.convertScaleAbs(imagen_procesada)

    return imagen_procesada

In [11]:
def paso_alto(ruta_imagen_a_procesar: str):
    # Visualización de la imagen de original
    img = Image.open(ruta_imagen_a_procesar)
    img.show()

    # Imagen procesada con algoritmo encontrado en el repositorio Github de la autora nishagandhi
    # https://github.com/nishagandhi/Histogram-Filters-Blending
    image_algoritmo_personalizado = cv2.imread(ruta_imagen_a_procesar, 0)
    imagen_procesada_algoritmo_personalizado = high_pass_filter(image_algoritmo_personalizado)
    guardar_mostrar_imagen(imagen_procesada_algoritmo_personalizado, "Implementacion de nishagandhi",
                           "Imagen_Procesada_Implementacion_Paso_Alto")

    # Implententación obtenida del libro Raspberry Pi Image Processing Programming Segunda Edicion
    escala_de_grises_scipy = ImageOps.grayscale(img)
    imagen_procesa_scipy = ndi.laplace(escala_de_grises_scipy)
    guardar_mostrar_imagen(imagen_procesa_scipy, "Imagen Procesada con Scipy", "Imagen_Procesada_Con_Libreria_Scipy")

    # Implementación obtenida de la documentación oficial de la librería OpenCV
    # https://docs.opencv.org/4.x/d5/db5/tutorial_laplace_operator.html
    imagen_procesa_opencv = procesar_imagen_con_opencv(ruta_imagen_a_procesar)
    guardar_mostrar_imagen(imagen_procesa_opencv, "Imagen Procesada con OpenCV", "Imagen_Procesada_Con_Libreria_OpenCV")

In [12]:
image_1 = "img-fil-laplaciano/1.jpg"
image_2 = "img-fil-laplaciano/2.jpg"

paso_alto(image_2)